In [1]:
import numpy as np
import pandas as pd

## Define Matrices

## M_a

In [2]:
M_a = np.ones((4, 4)) * 0.01 + np.diag([8, 4, 1, 1])
M_a.round(decimals=3)

array([[8.01, 0.01, 0.01, 0.01],
       [0.01, 4.01, 0.01, 0.01],
       [0.01, 0.01, 1.01, 0.01],
       [0.01, 0.01, 0.01, 1.01]])

## M_b

In [3]:
g_ba = np.array([
    [1., 0, 1, 0],
    [0, 1, 0, 1]
])

In [4]:
Lambda_col_a, V_a = np.linalg.eig(M_a)
Lambda_col_a, V_a.round(decimals=3)

(array([8.01005376, 1.01990469, 4.01004155, 1.        ]),
 array([[ 1.   ,  0.002, -0.003, -0.   ],
        [ 0.003,  0.005,  1.   ,  0.   ],
        [ 0.001, -0.707,  0.003, -0.707],
        [ 0.001, -0.707,  0.003,  0.707]]))

In [5]:
n_b, n_a = g_ba.shape

idx = Lambda_col_a.argsort()[-n_b:][::-1]   # -n_b
Lambda_col_a_top_b = Lambda_col_a[idx]  # top 1 eigenvector... # n_b eigenvalues
V_a_top_b = V_a[:,idx]           # corresponding eigenvectors

Lambda_col_a_top_b, V_a_top_b.round(decimals=3)

(array([8.01005376, 4.01004155]),
 array([[ 1.   , -0.003],
        [ 0.003,  1.   ],
        [ 0.001,  0.003],
        [ 0.001,  0.003]]))

In [6]:
(g_ba @ V_a_top_b).round(decimals=3)

array([[1.001e+00, 1.000e-03],
       [4.000e-03, 1.003e+00]])

In [7]:
np.linalg.inv(g_ba @ V_a_top_b).round(decimals=3)

array([[ 0.999, -0.001],
       [-0.004,  0.997]])

### Finally, M_b:

In [8]:
M_b = g_ba @ M_a @ V_a_top_b @ np.linalg.inv(g_ba @ V_a_top_b)
M_b.round(decimals=3)

array([[ 8.01e+00, -3.00e-03],
       [ 1.60e-02,  4.01e+00]])

In [30]:
M_b @ M_b @ M_b @ np.array([[1, 1.]]).T

array([[513.56717616],
       [ 66.25065799]])

## M_c

In [9]:
g_cb = np.array([
    [1., 1.],
])

In [10]:
Lambda_col_b, V_b = np.linalg.eig(M_b)
Lambda_col_b, V_b.round(decimals=3)

(array([8.01005376, 4.01004155]),
 array([[1.   , 0.001],
        [0.004, 1.   ]]))

In [11]:
n_c, n_b = g_cb.shape

idx = Lambda_col_b.argsort()[-n_c:][::-1]
Lambda_col_b_top_c = Lambda_col_b[idx]  # top n_b eigenvalues
V_b_top_c = V_b[:,idx]           # corresponding eigenvectors

Lambda_col_b_top_c, V_b_top_c.round(decimals=3)

(array([8.01005376]),
 array([[1.   ],
        [0.004]]))

In [12]:
(g_cb @ V_b_top_c).round(decimals=3)

array([[1.004]])

In [13]:
np.linalg.inv(g_cb @ V_b_top_c).round(decimals=3)

array([[0.996]])

## Finally, M_c

In [14]:
M_c = g_cb @ M_b @ V_b_top_c @ np.linalg.inv(g_cb @ V_b_top_c)
M_c.round(decimals=3)

array([[8.01]])

# Evolving in time

## Initial Conditions

In [43]:
N_a_0 = np.array([[1., 0., 1., 0.]]).T
N_a_0

array([[1.],
       [0.],
       [1.],
       [0.]])

In [44]:
N_b_0 = g_ba @ N_a_0
N_b_0

array([[2.],
       [0.]])

In [45]:
N_c_0 = g_cb @ N_b_0
N_c_0

array([[2.]])

## Time Series N_a

In [46]:
N_a_t = [(np.linalg.matrix_power(M_a, t) @ N_a_0).round(decimals=3) for t in range(10)]
N_a_t


# What best subspace to choose?

# Strictly required:
# Additional restriction on V such that always get positive entries of M_b
# Only allocate cases from a coarse-grained region into the fine grained regions that it includes
# (i.e. N^b = g_ba N^a)
# g_ba^{-1} should start with g_ba.T, only gets to modify entries that have 1's in them, have each column sum to 1.

# Desired:
# So long-term trajectory stays the same.
# Don't stray too far... How far off will we be?

# Questions:
# Does 2nd largest eigenvalue need to be here? [Can't be there exactly...]

# Ideas:
# Split cases only using top 1 eigenvector
# Split cases using 2x2 sub-matrices within the coarse-grained region

[array([[1.],
        [0.],
        [1.],
        [0.]]),
 array([[8.02],
        [0.02],
        [1.02],
        [0.02]]),
 array([[64.251],
        [ 0.171],
        [ 1.111],
        [ 0.111]]),
 array([[514.663],
        [  1.34 ],
        [  1.767],
        [  0.767]]),
 array([[4122.488],
        [  10.544],
        [   6.953],
        [   5.953]]),
 array([[33021.364],
        [   83.635],
        [   48.412],
        [   47.412]]),
 array([[264502.917],
        [   666.548],
        [   380.42 ],
        [   379.42 ]]),
 array([[2118682.628],
        [   5325.485],
        [   3039.713],
        [   3038.713]]),
 array([[16970761.886],
        [   42602.807],
        [   24340.579],
        [   24339.579]]),
 array([[1.35936716e+08],
        [3.41031675e+05],
        [1.94961027e+05],
        [1.94960027e+05]])]

## Time Series N_b

In [47]:
N_b_t = [(g_ba @ np.linalg.matrix_power(M_a, t) @ N_a_0).round(decimals=3) for t in range(10)]
N_b_t

[array([[2.],
        [0.]]),
 array([[9.04],
        [0.04]]),
 array([[65.362],
        [ 0.282]]),
 array([[516.43 ],
        [  2.107]]),
 array([[4129.441],
        [  16.496]]),
 array([[33069.776],
        [  131.047]]),
 array([[264883.337],
        [  1045.968]]),
 array([[2121722.341],
        [   8364.199]]),
 array([[16995102.464],
        [   66942.385]]),
 array([[1.36131677e+08],
        [5.35991702e+05]])]

## Time Series N_c

In [48]:
N_c_t = [(g_cb @ g_ba @ np.linalg.matrix_power(M_a, t) @ N_a_0).round(decimals=3) for t in range(10)]
N_c_t

[array([[2.]]),
 array([[9.08]]),
 array([[65.643]]),
 array([[518.537]]),
 array([[4145.937]]),
 array([[33200.822]]),
 array([[265929.305]]),
 array([[2130086.539]]),
 array([[17062044.85]]),
 array([[1.36667668e+08]])]

## Time Series N_b_hat

In [49]:
N_b_hat_t = [(np.linalg.matrix_power(M_b, t) @ N_b_0).round(decimals=3) for t in range(10)]
N_b_hat_t

[array([[2.],
        [0.]]),
 array([[16.02 ],
        [ 0.031]]),
 array([[128.322],
        [  0.378]]),
 array([[1027.868],
        [   3.538]]),
 array([[8233.283],
        [  30.368]]),
 array([[65949.043],
        [  251.393]]),
 array([[528255.41 ],
        [  2046.316]]),
 array([[4231354.335],
        [  16522.024]]),
 array([[33893376.109],
        [  132867.296]]),
 array([[2.71487766e+08],
        [1.06637943e+06]])]

## Time Series N_c_hat

In [38]:
N_c_hat_t = [(np.linalg.matrix_power(M_c, t) @ N_c_0).round(decimals=3) for t in range(10)]
N_c_hat_t

[array([[2.]]),
 array([[16.02]]),
 array([[128.322]]),
 array([[1027.865]]),
 array([[8233.258]]),
 array([[65948.838]]),
 array([[528253.739]]),
 array([[4231340.843]]),
 array([[33893267.612]]),
 array([[2.71486896e+08]])]

## An alternate M_b construction

In [39]:
M_a

array([[8.01, 0.01, 0.01, 0.01],
       [0.01, 4.01, 0.01, 0.01],
       [0.01, 0.01, 1.01, 0.01],
       [0.01, 0.01, 0.01, 1.01]])

In [40]:
g_ba

array([[1., 0., 1., 0.],
       [0., 1., 0., 1.]])

In [41]:
g_ba @ M_a @ g_ba.T

array([[9.04, 0.04],
       [0.04, 5.04]])

In [42]:
g_ba @ M_a @ np.linalg.pinv(g_ba)

array([[4.52, 0.02],
       [0.02, 2.52]])